In [ ]:
import logging
import warnings
logging.getLogger('pgmpy').setLevel(logging.ERROR)
logging.getLogger('pgmpy').setLevel(logging.WARNING)
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=RuntimeWarning, module="networkx.utils.backends")

import os
import random
import itertools
import pandas as pd
import numpy as np
from time import *
import networkx as nx

from copy import deepcopy
os.environ["NUMEXPR_MAX_THREADS"] = "16"
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import  TabularCPD

from Decom_Tree import *
from BeliefPropagation import *
from DiscreteMarkovNetwork import *
from Inference_decom import *

In [ ]:
def generate_state_names(g):
    return {node: [0, 1] for node in list(g.nodes)}

def get_random_cpds_with_labels(model, state_names, inplace=False, seed=None):
    cpds = []
    for node in model.nodes():
        parents = list(model.predecessors(node))
        cpds.append(
            TabularCPD.get_random(
                variable=node,
                evidence=parents,
                cardinality={var: 2 for var in model.nodes()},  
                state_names=state_names, 
                seed=seed
            )
        )
    if inplace:
        model.add_cpds(*cpds)
    else:
        return cpds

In [ ]:
model = get_example_model('pathfinder')#
G = nx.DiGraph()
G.add_nodes_from(model.nodes)
G.add_edges_from(model.edges)
state_names = generate_state_names(G)

bn = DiscreteBayesianNetwork()
bn.add_nodes_from(list(G.nodes))
bn.add_edges_from(list(G.edges))

learn_bn = DiscreteBayesianNetwork()
learn_bn.add_nodes_from(list(G.nodes))
learn_bn.add_edges_from(list(G.edges))

In [ ]:
#3
logging.getLogger('pgmpy').setLevel(logging.ERROR)

avg_decom_T,avg_full_T,avg_lossless_T = [],[],[]
avg_decom_RMSE,avg_full_RMSE,avg_lossless_RMSE = [],[],[]

rep_num = 100
sample_size = 1000
R_size=[2,5,10,15,20,25] #
for r_size in R_size:
    print(r_size)
    
    bn.cpds = []
    get_random_cpds_with_labels(bn, state_names, inplace=True)
    ori_infer = BeliefPropagation(bn) 
    
    df = BayesianModelSampling(bn).forward_sample(size=sample_size, show_progress=False)
    
    decom_T,full_T,lossless_T = 0,0,0
    decom_RMSE,full_RMSE,lossless_RMSE = 0,0,0
    for i in range(rep_num):
        R = list(random.sample(list(G.nodes), r_size))
        ori_value = ori_infer.query(variables = R,  show_progress=False) #真值
        

        #极小分解
        infer_engine = Inference_decom(G, df, R, method="decom") 
        start = time()
        marginal_result_decom = infer_engine.query()
        decom_T += (time()-start)
        decom_RMSE += np.mean((marginal_result_decom - np.array(ori_value.values)) ** 2)
        
        
        #无损分解
        infer_engine = Inference_decom(G, df, R, method="lossless") 
        start = time()
        marginal_result_lossless = infer_engine.query()
        lossless_T += (time()-start)
        lossless_RMSE += np.mean((marginal_result_lossless - np.array(ori_value.values)) ** 2)

        
        #传统分解
        learn_bn.cpds = []
        learn_bn.fit(df,estimator=MaximumLikelihoodEstimator)
        full_infer = BeliefPropagation(learn_bn) 
        start = time()
        marginal_result_full = full_infer.query(variables = R, show_progress=False) 
        full_T += time()-start
        full_RMSE += np.mean((np.array(marginal_result_full.values) - np.array(ori_value.values)) ** 2)
      
    
    avg_decom_T.append(decom_T/rep_num)
    avg_lossless_T.append(lossless_T/rep_num)
    avg_full_T.append(full_T/rep_num)
    
    avg_decom_RMSE.append(decom_RMSE/rep_num)
    avg_lossless_RMSE.append(lossless_RMSE/rep_num)
    avg_full_RMSE.append(full_RMSE/rep_num)
    

results_df = pd.DataFrame({
    "R_dim": R_size,
    "Decom_Time": avg_decom_T,
    "Decom_RMSE": avg_decom_RMSE,
    "Lossless_Time": avg_lossless_T,
    "Lossless_RMSE": avg_lossless_RMSE,
    "BP_Time": avg_full_T,
    "BP_RMSE": avg_full_RMSE
})

results_df